In [1]:
import os
import sys
import random
import argparse
import numpy as np
from PIL import Image, ImageFile

In [6]:
os.path.dirname(os.path.abspath('no_mask.jpg'))

'C:\\Users\\USER\\Downloads\\jupyter_notebook'

In [22]:
IMAGE_DIR = os.path.join(os.path.dirname(os.path.abspath('no_mask.jpg')), 'images')
DEFAULT_IMAGE_PATH = os.path.join(IMAGE_DIR, 'default-mask.png')
BLACK_IMAGE_PATH = os.path.join(IMAGE_DIR, 'black-mask.png')
BLUE_IMAGE_PATH = os.path.join(IMAGE_DIR, 'blue-mask.png')
RED_IMAGE_PATH = os.path.join(IMAGE_DIR, 'red-mask.png')
NOSE_IMAGE_PATH = os.path.join(IMAGE_DIR, 'nose_mask.png')
CHIN_IMAGE_PATH = os.path.join(IMAGE_DIR, 'chin_mask.png')

In [23]:
DEFAULT_IMAGE_PATH

'C:\\Users\\USER\\Downloads\\jupyter_notebook\\images\\default-mask.png'

In [24]:
def cli():
    parser = argparse.ArgumentParser(description='Wear a face mask in the given picture.')
    parser.add_argument('pic_path', help='Picture path.')
    parser.add_argument('--show', action='store_true', help='Whether show picture with mask or not.')
    parser.add_argument('--model', default='hog', choices=['hog', 'cnn'], help='Which face detection model to use.')
    group = parser.add_mutually_exclusive_group()
    group.add_argument('--black', action='store_true', help='Wear black mask')
    group.add_argument('--blue', action='store_true', help='Wear blue mask')
    group.add_argument('--red', action='store_true', help='Wear red mask')
    args = parser.parse_args()

    pic_path = args.pic_path
    if not os.path.exists(args.pic_path):
        print(f'Picture {pic_path} not exists.')
        sys.exit(1)

    if args.black:
        mask_path = BLACK_IMAGE_PATH
    elif args.blue:
        mask_path = BLUE_IMAGE_PATH
    elif args.red:
        mask_path = RED_IMAGE_PATH
    else:
        mask_path = DEFAULT_IMAGE_PATH

    FaceMasker(pic_path, mask_path, args.show, args.model).mask()

In [121]:
def create_mask(image_path):
    pic_path = image_path
    mask_path = "images/nose_mask.png"
    show = False
    model = "hog"
    FaceMasker(pic_path, mask_path, show, model).mask()

In [130]:
class FaceMasker:
    KEY_FACIAL_FEATURES = ('nose_bridge', 'chin', 'top_lip', 'nose_tip')

    def __init__(self, face_path, mask_path, show=False, model='hog'):
        self.face_path = face_path
        self.mask_path = mask_path
        self.show = show
        self.model = model
        self._face_img: ImageFile = None
        self._mask_img: ImageFile = None

    def mask(self):
        import face_recognition

        face_image_np = face_recognition.load_image_file(self.face_path)
        face_locations = face_recognition.face_locations(face_image_np, model=self.model)
        face_landmarks = face_recognition.face_landmarks(face_image_np, face_locations)
        self._face_img = Image.fromarray(face_image_np)
        self._mask_img = Image.open(self.mask_path)

        found_face = False
        for face_landmark in face_landmarks:
            # check whether facial features meet requirement
            skip = False
            for facial_feature in self.KEY_FACIAL_FEATURES:
                if facial_feature not in face_landmark:
                    skip = True
                    break
            if skip:
                continue

            # mask face
            found_face = True
            self._mask_face(face_landmark)

        if found_face:
            if self.show:
                self._face_img.show()

            # save
            self._save()
        else:
            print('Found no face.')

    def _mask_face(self, face_landmark: dict):
        nose_tip = face_landmark['nose_tip']
        nose_tip_point = nose_tip[len(nose_tip) // 2]
        nose_tip_v = np.array(nose_tip_point)
        
        top_lip = face_landmark['top_lip']
        top_lip_point = top_lip[len(top_lip) // 2]
        top_lip_v = np.array(top_lip_point)
        
        nose_bridge = face_landmark['nose_bridge']
        nose_point = nose_bridge[len(nose_bridge) * 1 // 4]
        nose_v = np.array(nose_point)

        chin = face_landmark['chin']
        chin_len = len(chin)
        chin_bottom_point = chin[chin_len // 2]
        chin_bottom_v = np.array(chin_bottom_point)
        chin_left_point = chin[chin_len // 8]
        chin_right_point = chin[chin_len * 7 // 8]

        # split mask and resize
        width = self._mask_img.width
        height = self._mask_img.height
        width_ratio = 1.2
#         new_height = int(np.linalg.norm(nose_v - chin_bottom_v))
        new_height = int(np.linalg.norm(nose_tip_v - chin_bottom_v))

        # left
        mask_left_img = self._mask_img.crop((0, 0, width // 2, height))
#         mask_left_width = self.get_distance_from_point_to_line(chin_left_point, nose_point, chin_bottom_point)
        mask_left_width = self.get_distance_from_point_to_line(chin_left_point, nose_tip_point, chin_bottom_point)
        mask_left_width = int(mask_left_width * width_ratio)
        mask_left_img = mask_left_img.resize((mask_left_width, new_height))

        # right
        mask_right_img = self._mask_img.crop((width // 2, 0, width, height))
#         mask_right_width = self.get_distance_from_point_to_line(chin_right_point, nose_point, chin_bottom_point)
        mask_right_width = self.get_distance_from_point_to_line(chin_right_point, nose_tip_point, chin_bottom_point)
        mask_right_width = int(mask_right_width * width_ratio)
        mask_right_img = mask_right_img.resize((mask_right_width, new_height))

        # merge mask
        size = (mask_left_img.width + mask_right_img.width, new_height)
        mask_img = Image.new('RGBA', size)
        mask_img.paste(mask_left_img, (0, 0), mask_left_img)
        mask_img.paste(mask_right_img, (mask_left_img.width, 0), mask_right_img)

        # rotate mask
#         angle = np.arctan2(chin_bottom_point[1] - nose_point[1], chin_bottom_point[0] - nose_point[0])
        angle = np.arctan2(chin_bottom_point[1] - nose_tip_point[1], chin_bottom_point[0] - nose_tip_point[0])
        rotated_mask_img = mask_img.rotate(angle, expand=True)

        # calculate mask location
#         center_x = (nose_point[0] + chin_bottom_point[0]) // 2
        center_x = (nose_tip_point[0] + chin_bottom_point[0]) // 2
#         center_y = (nose_point[1] + chin_bottom_point[1]) // 2
        center_y = (nose_tip_point[1] + chin_bottom_point[1]) // 2

        offset = mask_img.width // 2 - mask_left_img.width
        radian = angle * np.pi / 180
        box_x = center_x + int(offset * np.cos(radian)) - rotated_mask_img.width // 2
        box_y = center_y + int(offset * np.sin(radian)) - rotated_mask_img.height // 2

        # add mask
        self._face_img.paste(mask_img, (box_x, box_y), mask_img)

    def _save(self):
        path_splits = os.path.splitext(self.face_path)
#         new_face_path = path_splits[0] + '-with-mask' + path_splits[1]
#         new_face_path = path_splits[0] + '-with-mask(nose_tip)' + path_splits[1]
#         new_face_path = path_splits[0] + '-with-chin-mask' + path_splits[1]
#         new_face_path = path_splits[0] + '-with-chin-mask(nose_tip)' + path_splits[1]
#         new_face_path = path_splits[0] + '-with-nose-mask' + path_splits[1]
        new_face_path = path_splits[0] + '-with-nose-mask(nose_tip)' + path_splits[1]
        self._face_img.save(new_face_path)
        print(f'Save to {new_face_path}')

    @staticmethod
    def get_distance_from_point_to_line(point, line_point1, line_point2):
        distance = np.abs((line_point2[1] - line_point1[1]) * point[0] +
                          (line_point1[0] - line_point2[0]) * point[1] +
                          (line_point2[0] - line_point1[0]) * line_point1[1] +
                          (line_point1[1] - line_point2[1]) * line_point1[0]) / \
                   np.sqrt((line_point2[1] - line_point1[1]) * (line_point2[1] - line_point1[1]) +
                           (line_point1[0] - line_point2[0]) * (line_point1[0] - line_point2[0]))
        return int(distance)

In [133]:
if __name__ == '__main__':
    #cli()
    create_mask('2인이상2.jpg')

Save to 2인이상2-with-nose-mask(nose_tip).jpg


In [94]:
import face_recognition
face_image_np = face_recognition.load_image_file('장나라.jpg')
face_locations = face_recognition.face_locations(face_image_np, model='hog')
face_landmarks = face_recognition.face_landmarks(face_image_np, face_locations)

In [95]:
face_locations

[(82, 527, 350, 260)]

In [57]:
KEY_FACIAL_FEATURES = ('nose_bridge', 'chin', 'top_lip', 'nose_tip')
for face_landmark in face_landmarks:
            # check whether facial features meet requirement
    skip = False
    for facial_feature in KEY_FACIAL_FEATURES:
        if facial_feature not in face_landmark:
            skip = True
            break
    if skip:
        continue

In [42]:
nose_bridge = face_landmark['nose_bridge']
nose_point = nose_bridge[len(nose_bridge) * 1 // 4]
nose_v = np.array(nose_point)

In [43]:
nose_bridge

[(80, 61), (80, 66), (80, 72), (80, 77)]

In [47]:
chin = face_landmark['chin']
chin_len = len(chin)
chin_bottom_point = chin[chin_len // 2]
chin_bottom_v = np.array(chin_bottom_point)
chin_left_point = chin[chin_len // 8]
chin_right_point = chin[chin_len * 7 // 8]

In [52]:
top_lip = face_landmark['top_lip']
top_lip

[(69, 85),
 (74, 84),
 (78, 83),
 (82, 83),
 (85, 83),
 (91, 82),
 (97, 83),
 (95, 84),
 (86, 85),
 (82, 86),
 (78, 85),
 (71, 85)]

In [56]:
top_lip_point = top_lip[len(top_lip) // 2]
top_lip_point

(97, 83)

In [54]:
chin_bottom_point

(84, 107)

In [59]:
nose_tip = face_landmark['nose_tip']
nose_tip_point = nose_tip[len(nose_tip) // 2]
nose_tip_point

(81, 81)